In [1]:
import os
import time
import subprocess
import signal
import statistics
from pathlib import Path

import torch
import cv2
import urllib.parse
import numpy as np
import matplotlib.pyplot as plt
from prompt_toolkit.win32_types import INPUT_RECORD
from ultralytics import YOLO
from IPython.display import display, clear_output
from dotenv import load_dotenv


In [2]:
if not os.path.exists(os.path.join(os.getcwd(), ".env")):
    raise FileNotFoundError("'.env' file not found at current directory.")

load_dotenv()

HOST = os.getenv("HOST")
INPUT_STREAM_NAME = os.getenv("INPUT_STREAM_NAME")
OUT_STREAM_NAME = INPUT_STREAM_NAME + "_AI"

MAX_FRAMES = int(os.getenv("MAX_FRAMES"))


In [3]:
print(HOST, INPUT_STREAM_NAME, OUT_STREAM_NAME, MAX_FRAMES)


127.0.0.1 cam_09_стол cam_09_стол_AI 100


In [4]:
MODEL_PATH = (Path.cwd() / "from_GitHub/dmmmit_smoking_detection/models/final_model.pt").resolve()

print("Model path:", MODEL_PATH)
print("Exists:", MODEL_PATH.exists(), "Is file:", MODEL_PATH.is_file())

assert MODEL_PATH.is_file(), f"Expected a file, got: {MODEL_PATH}"


Model path: /home/jetson/GitHub/AI_Vision_Smoking/from_GitHub/dmmmit_smoking_detection/models/final_model.pt
Exists: True Is file: True


In [5]:
model = YOLO(str(MODEL_PATH))
model_type = "ultralytics.YOLO"


In [6]:
print("Task:", getattr(model, "task", None))
print("Class names:", getattr(model, "names", None))

core = getattr(model, "model", None)

# if core is not None:

#     print("Core type:", type(core))
#     if hasattr(core, "yaml"):
#         print("YAML:", core.yaml)

#     if hasattr(core, "stride"):
#         print("Stride:", core.stride)

#     if hasattr(core, "args"):
#         print("Args:", core.args)

#     n_params = sum(p.numel() for p in core.parameters())
#     print("Param count:", n_params)

#     print("Core module:", core)


Task: detect
Class names: {0: 'Person', 1: 'cell phone', 2: 'cigarette', 3: 'hands_with_cigarettes'}


In [7]:
# Input/output probe

DUMMY_H = 640
DUMMY_W = 640

dummy = np.zeros((DUMMY_H, DUMMY_W, 3), dtype=np.uint8)
print("Input dummy shape:", dummy.shape, "dtype:", dummy.dtype)

results = model.predict(dummy, verbose=False)
r = results[0]

print("Output: boxes.xyxy", r.boxes.xyxy.shape)
print("Output: boxes.conf", r.boxes.conf.shape)
print("Output: boxes.cls", r.boxes.cls.shape)

if r.masks is not None:
    print("Output: masks", r.masks.data.shape)


Input dummy shape: (640, 640, 3) dtype: uint8
Output: boxes.xyxy torch.Size([1, 4])
Output: boxes.conf torch.Size([1])
Output: boxes.cls torch.Size([1])


In [8]:
# go2rtc отдаёт rtsp на порту 8554

INPUT_STREAM_URL = f"rtsp://{HOST}:8554/{urllib.parse.quote(INPUT_STREAM_NAME)}?video"
print("STREAM_URL:", INPUT_STREAM_URL)

# иногда помогает для RTSP в OpenCV/FFMPEG
os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = "rtsp_transport;tcp|max_delay;500000|stimeout;5000000|loglevel;error"


STREAM_URL: rtsp://127.0.0.1:8554/cam_09_%D1%81%D1%82%D0%BE%D0%BB?video


In [9]:
OUT_STREAM_URL = f"rtsp://{HOST}:8554/{OUT_STREAM_NAME}"


In [10]:
def open_video_capture(stream_url: str) -> cv2.VideoCapture:
    video_capture = None

    try:
        video_capture = cv2.VideoCapture(stream_url, cv2.CAP_FFMPEG)
        video_capture.set(cv2.CAP_PROP_BUFFERSIZE, 1)
    except Exception as e:
        print(f"Exception: {e}")

    if not video_capture.isOpened():
        raise RuntimeError(f"RTSP stream not opened: {stream_url}")

    return video_capture


In [11]:
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    inter = max(0, xB - xA) * max(0, yB - yA)

    if inter == 0:
        return 0.0

    areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    return inter / (areaA + areaB - inter + 1e-9)


In [12]:
def smoking_score(results):
    # Эвристика: максимум из
    # - conf у класса hands_with_cigarettes
    # - conf у сигареты, которая пересекается с человеком
    names = results[0].names
    boxes = results[0].boxes

    persons = []
    cigarettes = []
    hands = []

    for b in boxes:
        cls_id = int(b.cls.item())
        conf = float(b.conf.item())
        name = names.get(cls_id, str(cls_id))
        xyxy = b.xyxy[0].cpu().numpy().tolist()

        if name == "Person":
            persons.append((xyxy, conf))

        elif name == "cigarette":
            cigarettes.append((xyxy, conf))

        elif name == "hands_with_cigarettes":
            hands.append(conf)

    score = max(hands) if hands else 0.0

    for c_box, c_conf in cigarettes:
        for p_box, _ in persons:
            if iou(c_box, p_box) > 0.01:
                score = max(score, c_conf)

    return score


In [13]:
def is_downscale_needed(h, w, max_w=1920, max_h=1080) -> bool:
    return w > max_w or h > max_h


In [14]:
def downscale_frame(frame, h, w, max_w=1920, max_h=1080) -> cv2.VideoCapture:
    scale = min(max_w / w, max_h / h)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(frame, (new_w, new_h), interpolation=cv2.INTER_AREA)
    return resized


In [15]:
def start_ffmpeg_proc(out_url: str, w: int, h: int, fps: int) -> subprocess.Popen:
    cmd = [
            "ffmpeg", "-hide_banner",
            "-loglevel", "error",
            "-f", "rawvideo",
            "-pix_fmt", "bgr24",
            "-s", f"{w}x{h}",
            "-r", str(fps),
            "-i", "-",
            "-an",
            "-c:v", "libx264",
            "-pix_fmt", "yuv420p",
            "-preset", "ultrafast",
            "-tune", "zerolatency",
            "-crf", "30",
            "-g", str(fps),
            "-bf", "0",
            "-f", "rtsp",
            "-rtsp_transport", "tcp",
            out_url
            ]

    return subprocess.Popen(
            cmd,
            stdin=subprocess.PIPE,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            bufsize=0
            )

In [16]:
def stop_ffmpeg_proc(ffmpeg_proc):
    if ffmpeg_proc is None:
        return

    try:
        if ffmpeg_proc.stdin:
            ffmpeg_proc.stdin.close()
    except Exception as e:
        print(f"Exception: {e}")
        pass

    try:
        ffmpeg_proc.send_signal(signal.SIGINT)
        ffmpeg_proc.wait(timeout=2)
    except Exception as e:
        ffmpeg_proc.kill()
        print(f"Exception: {e}")


In [17]:
def annotate_rtsp(
        input_stream_url: str,
        out_stream_url: str,
        target_fps: int,
        max_frames: int = 300,
        seconds: int | None = 15,
        num_frames_to_drop: int = 2,
        conf: float = 0.25,
        imgsz: int = 640,
        down_w: int = 1280,
        down_h: int = 720,
        print_every: int = 30,
        ):
    #
    video_capture = open_video_capture(input_stream_url)
    ffmpeg_proc = None

    period = 1.0 / target_fps
    next_t = time.perf_counter()

    misses = 0
    frames = 0
    t_start = time.perf_counter()
    sleep = 0

    dts = []

    try:
        while True:

            if frames >= max_frames:
                break

            if seconds is not None and (time.perf_counter() - t_start) >= seconds:
                break

            t_proc0 = time.perf_counter()

            # берём 1 кадр всегда

            if not video_capture.grab():
                break

            is_video_capture_retrieved, frame = video_capture.retrieve()

            if not is_video_capture_retrieved or frame is None:
                break

            # если мы начинаем отставать, тогда выбрасываем ещё кадры
            if sleep < -0.050:  # отстаём больше чем на 50мс
                for _ in range(num_frames_to_drop):
                    if not video_capture.grab():
                        break
                is_video_capture_retrieved, frame = video_capture.retrieve()

            h, w = frame.shape[:2]

            if is_downscale_needed(h, w, down_w, down_h):
                frame = downscale_frame(frame, h, w, down_w, down_h)

            predictions = model.predict(frame, conf=conf, imgsz=imgsz, verbose=False)

            annotated = frame
            names = predictions[0].names

            for b in predictions[0].boxes:
                x1, y1, x2, y2 = map(int, b.xyxy[0].tolist())
                cls_id = int(b.cls.item())
                conf_value = float(b.conf.item())
                name = names.get(cls_id, str(cls_id))

                cv2.rectangle(
                        img=annotated,
                        pt1=(x1, y1),
                        pt2=(x2, y2),
                        color=(0, 255, 0),
                        thickness=2
                        )

                cv2.putText(
                        img=annotated,
                        text=f"{name} {conf_value:.2f}",
                        org=(x1, max(20, y1 - 5)),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=0.6,
                        color=(0, 255, 0),
                        thickness=2,
                        lineType=cv2.LINE_AA
                        )

            smoke_prob = smoking_score(predictions)
            cv2.putText(
                    img=annotated,
                    text=f"smoking (heuristic): {smoke_prob:.2f}",
                    org=(10, 60),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.9,
                    color=(0, 0, 0),
                    thickness=2,
                    lineType=cv2.LINE_AA
                    )

            # старт энкодера один раз, после первого кадра
            if ffmpeg_proc is None:
                h, w = annotated.shape[:2]
                ffmpeg_proc = start_ffmpeg_proc(out_stream_url, w, h, target_fps)
                time.sleep(0.2)

            # пуш кадра
            try:
                ffmpeg_proc.stdin.write(annotated.tobytes())

            except BrokenPipeError:
                print("ffmpeg died, return code:", ffmpeg_proc.poll())
                break

            t_proc1 = time.perf_counter()
            proc_dt = t_proc1 - t_proc0
            proc_fps = 1.0 / proc_dt if proc_dt > 0 else 0.0

            # внутри цикла, сразу после t_proc1:
            dts.append(proc_dt)

            frames += 1

            if frames > 0 and frames % 120 == 0 and len(dts) >= 2:
                dts_sorted = sorted(dts)

                def pct(p: float) -> float:
                    # p in [0,100]
                    k = (len(dts_sorted) - 1) * (p / 100.0)
                    f = int(k)
                    c = min(f + 1, len(dts_sorted) - 1)
                    if f == c:
                        return dts_sorted[f]
                    return dts_sorted[f] + (dts_sorted[c] - dts_sorted[f]) * (k - f)

                p50 = pct(50)
                p90 = pct(90)
                p99 = pct(99)

                print(
                        f"proc_dt p50={p50 * 1000:.1f}ms "
                        f"p90={p90 * 1000:.1f}ms "
                        f"p99={p99 * 1000:.1f}ms "
                        f"max={max(dts_sorted) * 1000:.1f}ms "
                        f"n={len(dts_sorted)}"
                        )
                dts.clear()

            # pacing до target_fps
            next_t += period
            sleep = next_t - time.perf_counter()

            if sleep > 0:
                time.sleep(sleep)
            else:
                misses += 1
                next_t = time.perf_counter()

            if print_every and frames % print_every == 0:
                wall = time.perf_counter() - t_start
                out_fps = frames / wall if wall > 0 else 0.0
                print(f"target={target_fps} proc_fps≈{proc_fps:.2f} out_fps≈{out_fps:.2f} misses={misses}/{frames}")

        wall = time.perf_counter() - t_start
        out_fps = frames / wall if wall > 0 else 0.0
        return {
                "target" : target_fps,
                "out_fps": out_fps,
                "misses" : misses,
                "frames" : frames,
                }

    finally:

        try:
            video_capture.release()
        except Exception as e:
            print(f"Exception: {e}")

        stop_ffmpeg_proc(ffmpeg_proc)

In [18]:
for fps in [4, 5, 6, 7, 8, 10]:

    r = annotate_rtsp(
            input_stream_url=INPUT_STREAM_URL,
            out_stream_url=OUT_STREAM_URL,
            target_fps=fps,
            seconds=20,
            max_frames=10_000,
            num_frames_to_drop=2,
            conf=0.25,
            imgsz=640,
            down_w=1280,
            down_h=720,
            print_every=30,
            )
    print(r)

    if r["frames"] < 30:
        print("stop: stream too short / unstable")
        break

    if r["out_fps"] < fps * 0.90 or r["misses"] > r["frames"] * 0.10:
        print("stop: capacity reached")
        break


[h264 @ 0xaaaaef84baf0] non-existing PPS 0 referenced
[h264 @ 0xaaaaef84baf0] non-existing PPS 0 referenced
[h264 @ 0xaaaaef84baf0] decode_slice_header error
[h264 @ 0xaaaaef84baf0] no frame!
[h264 @ 0xaaaaef84baf0] non-existing PPS 0 referenced
[h264 @ 0xaaaaef84baf0] non-existing PPS 0 referenced
[h264 @ 0xaaaaef84baf0] decode_slice_header error
[h264 @ 0xaaaaef84baf0] no frame!
[h264 @ 0xaaaaef84baf0] non-existing PPS 0 referenced
[h264 @ 0xaaaaef84baf0] non-existing PPS 0 referenced
[h264 @ 0xaaaaef84baf0] decode_slice_header error
[h264 @ 0xaaaaef84baf0] no frame!
[h264 @ 0xaaaaef84baf0] non-existing PPS 0 referenced
[h264 @ 0xaaaaef84baf0] non-existing PPS 0 referenced
[h264 @ 0xaaaaef84baf0] decode_slice_header error
[h264 @ 0xaaaaef84baf0] no frame!
[h264 @ 0xaaaaef84baf0] non-existing PPS 0 referenced
[h264 @ 0xaaaaef84baf0] non-existing PPS 0 referenced
[h264 @ 0xaaaaef84baf0] decode_slice_header error
[h264 @ 0xaaaaef84baf0] no frame!
[h264 @ 0xaaaaef84baf0] non-existing PPS

target=4 proc_fps≈11.58 out_fps≈3.88 misses=1/30
target=4 proc_fps≈11.54 out_fps≈3.94 misses=1/60
{'target': 4, 'out_fps': 3.952968354719582, 'misses': 1, 'frames': 80}


[h264 @ 0xaaaae8e7e300] non-existing PPS 0 referenced
[h264 @ 0xaaaae8e7e300] non-existing PPS 0 referenced
[h264 @ 0xaaaae8e7e300] decode_slice_header error
[h264 @ 0xaaaae8e7e300] no frame!
[h264 @ 0xaaaae8e7e300] non-existing PPS 0 referenced
[h264 @ 0xaaaae8e7e300] non-existing PPS 0 referenced
[h264 @ 0xaaaae8e7e300] decode_slice_header error
[h264 @ 0xaaaae8e7e300] no frame!
[h264 @ 0xaaaae8e7e300] non-existing PPS 0 referenced
[h264 @ 0xaaaae8e7e300] non-existing PPS 0 referenced
[h264 @ 0xaaaae8e7e300] decode_slice_header error
[h264 @ 0xaaaae8e7e300] no frame!
[h264 @ 0xaaaae8e7e300] non-existing PPS 0 referenced
[h264 @ 0xaaaae8e7e300] non-existing PPS 0 referenced
[h264 @ 0xaaaae8e7e300] decode_slice_header error
[h264 @ 0xaaaae8e7e300] no frame!
[h264 @ 0xaaaae8e7e300] non-existing PPS 0 referenced
[h264 @ 0xaaaae8e7e300] non-existing PPS 0 referenced
[h264 @ 0xaaaae8e7e300] decode_slice_header error
[h264 @ 0xaaaae8e7e300] no frame!
[h264 @ 0xaaaae8e7e300] non-existing PPS

target=5 proc_fps≈13.65 out_fps≈4.90 misses=1/30
target=5 proc_fps≈11.60 out_fps≈4.95 misses=1/60
target=5 proc_fps≈12.31 out_fps≈4.97 misses=1/90
{'target': 5, 'out_fps': 4.968484372253501, 'misses': 1, 'frames': 100}
target=6 proc_fps≈12.31 out_fps≈5.83 misses=1/30
target=6 proc_fps≈12.38 out_fps≈5.92 misses=1/60
target=6 proc_fps≈12.40 out_fps≈5.94 misses=1/90
proc_dt p50=80.3ms p90=81.7ms p99=89.6ms max=309.8ms n=120
target=6 proc_fps≈12.51 out_fps≈5.96 misses=1/120
{'target': 6, 'out_fps': 5.957270510989617, 'misses': 1, 'frames': 120}
target=7 proc_fps≈13.39 out_fps≈6.76 misses=1/30
target=7 proc_fps≈13.31 out_fps≈6.88 misses=1/60
target=7 proc_fps≈13.24 out_fps≈6.92 misses=1/90
proc_dt p50=75.3ms p90=77.4ms p99=86.5ms max=293.5ms n=120
target=7 proc_fps≈13.15 out_fps≈6.94 misses=1/120
{'target': 7, 'out_fps': 6.947237725462975, 'misses': 1, 'frames': 139}


[h264 @ 0xaaaaef888a60] non-existing PPS 0 referenced
[h264 @ 0xaaaaef888a60] non-existing PPS 0 referenced
[h264 @ 0xaaaaef888a60] decode_slice_header error
[h264 @ 0xaaaaef888a60] no frame!
[h264 @ 0xaaaaef888a60] non-existing PPS 0 referenced
[h264 @ 0xaaaaef888a60] non-existing PPS 0 referenced
[h264 @ 0xaaaaef888a60] decode_slice_header error
[h264 @ 0xaaaaef888a60] no frame!
[h264 @ 0xaaaaef888a60] non-existing PPS 0 referenced
[h264 @ 0xaaaaef888a60] non-existing PPS 0 referenced
[h264 @ 0xaaaaef888a60] decode_slice_header error
[h264 @ 0xaaaaef888a60] no frame!


target=8 proc_fps≈13.92 out_fps≈7.58 misses=1/30
target=8 proc_fps≈14.06 out_fps≈7.79 misses=1/60
target=8 proc_fps≈13.90 out_fps≈7.86 misses=1/90
proc_dt p50=71.0ms p90=73.6ms p99=95.5ms max=331.8ms n=120
target=8 proc_fps≈14.05 out_fps≈7.89 misses=1/120
target=8 proc_fps≈14.15 out_fps≈7.91 misses=1/150
{'target': 8, 'out_fps': 7.91749731605755, 'misses': 1, 'frames': 159}


[h264 @ 0xaaaaef86d940] non-existing PPS 0 referenced
[h264 @ 0xaaaaef86d940] non-existing PPS 0 referenced
[h264 @ 0xaaaaef86d940] decode_slice_header error
[h264 @ 0xaaaaef86d940] no frame!


target=10 proc_fps≈15.60 out_fps≈9.36 misses=1/30
target=10 proc_fps≈16.13 out_fps≈9.67 misses=1/60
target=10 proc_fps≈15.63 out_fps≈9.78 misses=1/90
proc_dt p50=63.4ms p90=65.4ms p99=94.5ms max=304.6ms n=120
target=10 proc_fps≈15.77 out_fps≈9.83 misses=1/120
target=10 proc_fps≈15.55 out_fps≈9.87 misses=1/150
target=10 proc_fps≈15.71 out_fps≈9.89 misses=1/180
{'target': 10, 'out_fps': 9.897631804987347, 'misses': 1, 'frames': 198}


In [19]:
# !ffmpeg -re -f lavfi -i testsrc=size=640x360:rate=10 \
# -an -c:v libx264 -pix_fmt yuv420p -tune zerolatency -preset veryfast \
# -f rtsp -rtsp_transport tcp rtsp://127.0.0.1:8554/cam_11_annotated
